<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/knu_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 라이브러리 설치
!pip install PyPDF2 chromadb sentence-transformers pandas

In [27]:
# 벡터 DB에 한번에 저장 => 거의 자동화 바꿀점 2가지 밖에 없음
import os
import re
import unicodedata
import pandas as pd
import pytz
from datetime import datetime
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from chromadb import PersistentClient
KST = pytz.timezone('Asia/Seoul')

# 설정: 처음에 한 번만 설정하고 거의 변경하지 않는 항목
PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"

# 타겟 폴더 루트 설정 (PDF 또는 HTML 루트)          ################ 여기 2가지만 바꾸기(나머지 건들지 마시오)
ROOT_FOLDER = "/content/drive/MyDrive/pdf"  # 또는 /html, /pdf          변경
URL_MAPPING_FILENAME = "_pdf_url.xlsx"      # 또는 _html_url, _pdf_url  변경

# 학과별 폴더 순회
department_dirs = [d for d in os.listdir(ROOT_FOLDER) if os.path.isdir(os.path.join(ROOT_FOLDER, d))]

# 학과명 매핑
department_name_map = {
    "software_department": "소프트웨어학과",
    "computer_department": "컴퓨터공학과",
    # 필요시 추가
}

# 카테고리 추출 함수 (경로에서 마지막 디렉토리 이름 사용)
def extract_category(file_path, department_root):
    rel_path = os.path.relpath(file_path, start=department_root)
    parts = rel_path.split(os.sep)
    if len(parts) > 1:
        return parts[0]
    return "기타"

# 텍스트 전처리 함수
def clean_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[■◆●※▶▷▲→◇]', '', text)
    return text.strip()

# 텍스트 청크화 함수
def split_chunks(text, chunk_size=500):
    return [text[i:i+chunk_size].strip() for i in range(0, len(text), chunk_size)]

# 파일 읽기 함수 (PDF 및 Markdown)
def read_file(filepath):
    if filepath.endswith('.pdf'):
        reader = PdfReader(filepath)
        text = "".join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif filepath.endswith('.md'):
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
    else:
        text = ""
    return text

# 하위 모든 파일 경로 가져오기 함수 (폴더 안에 폴더가 있을 수 있음)
def get_all_files(root_dir):
    all_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if f.endswith(('.pdf', '.md')):
                all_files.append(os.path.join(dirpath, f))
    return all_files

# Chroma DB 연결
client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)
existing_ids = set(collection.get(limit=None)['ids'])

# 임베딩 모델 로딩
model = SentenceTransformer(EMBEDDING_MODEL_NAME)

total_added = 0

# 모든 학과 폴더 순회
for dept_dir in department_dirs:
    folder_path = os.path.join(ROOT_FOLDER, dept_dir)
    department_kor = department_name_map.get(dept_dir, dept_dir)

    # URL 매핑 불러오기
    url_mapping_file = os.path.join(folder_path, dept_dir + URL_MAPPING_FILENAME)
    if not os.path.exists(url_mapping_file):
        print(f"❌ URL 매핑 파일 없음: {url_mapping_file}")
        continue

    url_df = pd.read_excel(url_mapping_file)
    url_mapping = dict(zip(url_df['파일명'], url_df['URL']))

    file_paths = get_all_files(folder_path)

    for file_path in file_paths:
        filename = os.path.basename(file_path)
        base_name = unicodedata.normalize("NFC", os.path.splitext(filename)[0])

        raw_text = read_file(file_path)
        if not raw_text.strip():
            print(f"⚠️ {filename}: 텍스트 없음 → 스킵")
            continue

        cleaned_text = clean_text(raw_text)
        chunks = split_chunks(cleaned_text)
        embeddings = model.encode(chunks).tolist()

        ids = [f"{dept_dir}_{base_name}_chunk_{i}" for i in range(len(chunks))]

        new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []

        for chunk, emb, id_ in zip(chunks, embeddings, ids):
            if id_ not in existing_ids:
                new_chunks.append(chunk)
                new_embeddings.append(emb)
                new_ids.append(id_)
                new_metadatas.append({
                    "file_name": base_name,
                    "department": department_kor,
                    "category": extract_category(file_path, folder_path),
                    "type": "pdf" if filename.endswith('.pdf') else "markdown",
                    "url": url_mapping.get(base_name, "출처 URL 미등록"),
                    "date": datetime.now(KST).isoformat()
                })

        if new_chunks:
            collection.add(
                documents=new_chunks,
                embeddings=new_embeddings,
                metadatas=new_metadatas,
                ids=new_ids
            )
            print(f"✅ {filename}: {len(new_chunks)}개 저장 완료")
            total_added += len(new_chunks)
        else:
            print(f"⚠️ {filename}: 중복 청크만 존재 → 스킵")

print(f"\n🎉 총 저장된 신규 청크 수: {total_added}")


✅ 소프트웨어전공 졸업규정.pdf: 4개 저장 완료
✅ SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문.pdf: 20개 저장 완료
✅ 졸업논문계획서-샘플.pdf: 1개 저장 완료

🎉 총 저장된 신규 청크 수: 25


In [32]:
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

results = collection.get(limit=None, include=["documents", "metadatas", "embeddings"])

print(f"\n✅ 전체 저장된 벡터 수: {len(results['ids'])}")
print("=" * 60)

for i in range(len(results["ids"])):
    print(f"📌 ID: {results['ids'][i]}")

    # 문서 내용 일부
    doc = results["documents"][i]
    if doc:
        print(f"📄 문서 내용 (앞부분): {doc[:300]}{'...' if len(doc) > 300 else ''}")
    else:
        print("📄 문서 내용: ❌ 없음")

    # 메타데이터 출력
    metadata = results["metadatas"][i]
    print(f"📎 메타데이터: {metadata}")

    # 벡터 길이 및 일부 출력 (에러 방지용 조건 추가)
    embedding = results["embeddings"][i]
    if embedding is not None and len(embedding) > 0:
        print(f"🧠 벡터 길이: {len(embedding)}")
        print("🧠 벡터 앞 10개 값:", list(embedding[:10]))
    else:
        print("🧠 벡터: ❌ 없음")

    print("=" * 60)


✅ 전체 저장된 벡터 수: 25
📌 ID: software_department_소프트웨어전공 졸업규정_chunk_0
📄 문서 내용 (앞부분): - 7 -소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 의거 소프트웨어전공 , 컴퓨터소프트웨어공학전공 및 멀티미디어공학전공의 졸업논문에 관한 세부사항을 규정함을 목적으로 한다. 제2조(졸업논문 제출자격 ) ① 최종학기 등록을 필한 자라야 한다. ② 3~4학년 재학중 공모전 출품을 1회 하여야 한다(출품인 4인까지 인정), 제3조(졸업논문 계획서 제출) ① 졸업논문 제출자격자는 4학년 1학기(최종학기 직전학기 )...
📎 메타데이터: {'category': '규정자료실', 'date': '2025-04-01T01:04:59.369814+09:00', 'department': '소프트웨어학과', 'file_name': '소프트웨어전공 졸업규정', 'type': 'pdf', 'url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426'}
🧠 벡터 길이: 384
🧠 벡터 앞 10개 값: [np.float64(-0.02203826792538166), np.float64(0.06772523373365402), np.float64(0.019005445763468742), np.float64(0.010297110304236412), np.float64(-0.05260750278830528), np.float64(-0.033055711537599564), np.float64(0.015976473689079285), np.float64(-0.07992683351039886), np.float64(-0.05584532767534256), np.float64(-0.023645076900

In [23]:
# 완전 초기화 ------ 신중히 사용하기 바람 -------
from chromadb import PersistentClient

PERSIST_DIR = "/content/drive/MyDrive/chroma_index"
COLLECTION_NAME = "knu_collection"

client = PersistentClient(path=PERSIST_DIR)

# 컬렉션 삭제
client.delete_collection(name=COLLECTION_NAME)
print("🧨 컬렉션 전체 삭제 완료 (초기화됨)")

# 다시 생성
client.create_collection(name=COLLECTION_NAME)
print("✅ 새 빈 컬렉션 재생성 완료")

🧨 컬렉션 전체 삭제 완료 (초기화됨)
✅ 새 빈 컬렉션 재생성 완료
